# Forward Algorithm

This is a recursive algorithm based on the concept of $\alpha(t, z_t)$, being $z_t$ the hidden state at time $t$ (we assume there are $M$ possible hidden states), which actually is $$\alpha(t, z_t) = p(x_1,...,x_t, z_t)$$ It is useful for computing the probability of a given sequence of observations $$ p(x_1,...,x_T)$$ in a polynomial (in $T$, the length of the sequence) amount of computation time. Recall that, in this computation, $x_1,...,x_T$ is a fixed sequence, and hence each value of $x_t$ is given.

As a preliminar state, note that, by marginalizing $$ p(x_1,...,x_T) = \sum_{z_1}^M ...\sum_{z_T}^M p(x_1,...x_T,z_1,...,z_T) $$ Then we can use the Markov property on $z_t$ and the fact that the probability of an observation at $t$, $x_t$, depends only on the hidden state at $t$, $z_t$. These properties allow us to write $$ p(x_1,...,x_T) = \sum_{z_1}^M ...\sum_{z_T}^M p(z_1) \prod_t^T p(z_{t+1}|z_{t}) p(x_t|z_t) $$ As an example, consider the case $M=2, T=2$: $$ p(x_1, x_2) = \sum_{z_1}^M \sum_{z_2}^M p(x_2, x_1, z_1, z_2) = \sum_{z_1}^M \sum_{z_2}^M p(x_1 | z_2, x_2, z_1)p(x_2, z_2, z_1) \\ = \sum_{z_1}^M \sum_{z_2}^M p(x_1|z_1)p(x_2|z_2)p(z_2|z_1)p(z_1) $$ 

Note that the expression for $p(x_1,...,x_T)$ can be written in terms of transition matrices $A_{ij} = p(z_{t+1}=j| z_t =i), B_{jk} = p(x_t=k| z_t = j)$ and initial probabilities $\pi_j = p(z_1=j)$: $$ p(x_1,...,x_T) = \sum_{z_1}^M ...\sum_{z_T}^M \pi_{z_1} \prod_t^T A_{z_{t+1}, z_t} B_{z_t, x_t}$$

Note that the transition probabilities do not depend themselves on time, only on the pair of states they are transitioning from/to. For example, if $M=2$, then $A(1, 2) \ne A(2,1)$ in general, but $A(1, 2)$ will be the same regardless of whether $t=3$ or $t=100$. 

Comming back to $\alpha$, note that $$ \alpha(t, z_t) = p(x_1,...,x_t, z_t) = p(x_t|z_t)p(x_1,...,x_{t-1}, z_t) =p(x_t|z_t) \sum_{z_{t-1}}^M p(z_t|z_{t-1}) \alpha(t-1, z_{t-1}) $$ This defines a recursive relations for the $\alpha$ s, which will allow the forward algorithm to scale polynomially with $T$ and not exponentially, in particular $\mathcal{O}(TM^2)$. Also, for the recursion to work, the initial value is $\alpha(t=1, z_1) =p(x_1|z_1) p(z_1) $. One can show that the original goal $p(x_1,...,x_T)$ can be written in terms of $\alpha$ as $$p(x_1,...,x_T) = \sum_{z_T}^M \alpha(t=T, z_T)$$

Consider a couple of explicit examples: $$ p(x_1, x_2) = \sum_{z_1}\sum_{z_2} p(x_1, x_2, z_1, z_2) = \sum_{z_2} p(x_2|z_2)\sum_{z_1} p(z_2|z_1) p(x_1|z_1)p(z_1)  = \sum_{z_2} \alpha(t=2, z_2)\,,$$ where, as shown above $$ \alpha(t=2, z_2) =  p(x_2|z_2)\sum_{z_1} p(z_2|z_1) p(x_1|z_1)p(z_1) =  p(x_2|z_2)\sum_{z_1} p(z_2|z_1) \alpha(t=1, z_1)$$

In conclusion, in terms of the original HMM parameters, the forward algorithm can be written as follows:

Initialization step: $\alpha(t=1, i) = \pi_i B(i, x_1)$ for $t=1$

Induction step: $\alpha(t+1, i) = B(i, x_{t+1}) \sum_j^M A(j,i) \alpha(t, j)$ for $t= 1, ..., T-1$

Termination step: $ p(x_1,...,x_T) = \sum_{i}^M \alpha(T, i) $ for $t=T$

The above notation is a bit misleading: $\alpha(t=1, i)$, for arbitrary $x_1$, will be a matrix of dimension $M \times K$, so the indices are exchanged with respect to matrix notation, as $i \in 1,...,M$ corresponds to the *row* and the hidden index referring to $x$ is in the *column*. This in particular implies that $$\sum_j^M A(j,i) \alpha(t, j)$$ is not a straightforward matrix multiplication: it involves the multiplication of a row of $A$ with a **row** of $\alpha$

In [1]:
import numpy as np
from typing import List, Union

Assume there exist matrices $A$ and $B$ and also vector $\pi$. Recall that $\dim(A) = M \times M$, $\dim(B) = M \times K$ and $\dim(\pi) = M \times 1$

Let's consider, as an example, the case $M = 3, K=2$.

In order to implement the algorith, note that the dependence on $x_t$ is implicit. If we want to compute $$p(x_1,...,x_T)$$ for an *arbitrary* sequence $x_1,...,x_T$, then we would have to consider a tensor with $T$ dimensions, one direction for each $x$. That is why it is in principle simpler to start from a fixed sequence. For example, if $K=2$, each $x_t \in \{1,2\}$. If we consider $T=3$ and fixed $x_1=2, x_2=2, x_3 = 1$, then we know that $x_1$ will be associated to the second column of $B$, $x_2$ will involve another factor with the second column of $B$ and $x_3$ will involve a third product with the first column of $B$. This makes it easier to perform the computations. Below, however, we will do the full tensor computation.

In [2]:
pi = np.array([0.1, 0.1, 0.8]).reshape(3,1) # 3x1
B = np.array([[0.1, 0.9],[0.35, 0.65],[0., 1.]]) # 3x2
A = np.array([[0.25, 0.25, 0.5], [0.4, 0.2, 0.4], [0.75, 0.2, 0.05]]) #3x3

In [3]:
# assume there exist matrices A and B and also vector \pi
# dim(A) = M x M, dim(B) = M x K, and dim(\pi) = M x 1
# for p(x1, ..., xT), the xs are fixed. This boilds down to picking a column of the resulting matrix
def alpha_initial(pi: Union[List, np.ndarray], B: Union[List, np.ndarray]) -> np.ndarray:
    '''
    Computes the value of alpha associated to initial time t=1
    
    Parameters
        pi: Union[List, np.ndarray]
            Initial vector of probabilities (i.e. p(z_1)) for each possible state of z_1
        B:  Union[List, np.ndarray]
            time-independent matrix of transition probabiltiies p(x_t|z_t) for each x- and z-states

    Return
        np.ndarray: initial value of alpha (namely, alpha_1)
    '''
    assert np.isclose(np.sum(pi), 1), "the sum of probabilities of initial latent states must add up to 1"
    assert (np.isclose(B.sum(axis=1), 1)).all(), "the total probability of a given observed state must be 1"
    return np.multiply(pi, B) # element-wise product. Note np.multiply(pi, B).sum()=1 as expected

def alpha_recurrent(
                    A: Union[List, np.ndarray], 
                    B: Union[List, np.ndarray], 
                    sequence_length: int, 
                    alpha_ini: np.ndarray
                    ) -> List[np.ndarray]:
    '''
    Computes a set of alphas following the (recursive) forward algorithm

    Parameters
        A:  Union[List, np.ndarray]
            Initial vector of probabilities (i.e. p(z_1)) for each possible state of z_1
        B:  Union[List, np.ndarray]
            time-independent matrix of transition probabiltiies p(x_t|z_t) for each x- and z-states
        sequence_length: int
            The length of the input sequence of xs whose join probability we want to compute
        alpha_ini: np,ndarray
            The value of alpha(t=1, z_1)
    Return
        List[np.ndarray]: list of alphas of the form 
        [alpha(t=1, z_1),..., alpha(t=T-1, z_{T-1}), alpha(t=T, z_T)]
        (note that each alpha will have a different dimensionality as a tensor: (M, K,...,K))
    '''
    A = np.array(A)
    B = np.array(B)
    assert isinstance(sequence_length, int), 'the length of the sequence must be an integer number'
    assert alpha_ini.shape[0] == A.shape[0], 'the length of the first axis of alpha_ini must be the rows of A'

    alphas_list = [alpha_ini] 
    for t_ in range(1,sequence_length):
        # this gives alpha allowing column of A to be different from row of B
        # elipsis ... stand for "any untouched extra dimension"
        alpha_expl = np.einsum('ik,jl...,js->ikl...s', B, alphas_list[t_-1], A)
        # force (column of A) = (row of B) = z_t = i  
        # i.e pick diagonal for first and last indices: i = s above, without summing over i!
        alpha_expl = np.diagonal(alpha_expl, axis1=0, axis2=len(alpha_expl.shape)-1)
        # np.diagonal() places the dim = M along the last direction of alpha_expl
        alphas_list.append(np.einsum('ij...k->kij...',alpha_expl))
    return alphas_list

def get_general_probability_from_alpha(alpha: np.ndarray) -> np.ndarray:
    '''
    Computes the probability of a generic sequence of x states given an alpha 
    (this alpha is suppossed to correspond to alpha(t=T, z_T))

    Parameters:
        alpha: np.ndarray
            A tensor with dimensions (M, K, K,...,K)
    Returns:
        np.ndarray:
            A tensor with dimension (K, K,...,K)
    '''
    # sum alpha along the first direction (that would correspond to z_T)
    # returns probability for arbitrary set of x
    # (note the ordering: ijk... will correspond to x_T, x_{T-1}, x_{T-2},...)
    return np.einsum('i...->...', alpha)

def get_probability_sequence(
                            pi: Union[List, np.ndarray], 
                            A: Union[List, np.ndarray], 
                            B: Union[List, np.ndarray], 
                            xs_sequence: Union[List, np.ndarray]
                            ) -> float:
    ''' 
    Computes the probability of a specific sequence of x states

    Parameters:
        pi: Union[List, np.ndarray]
            Initial vector of probabilities (i.e. p(z_1)) for each possible state of z_1
        A:  Union[List, np.ndarray]
            Initial vector of probabilities (i.e. p(z_1)) for each possible state of z_1
        B:  Union[List, np.ndarray]
            time-independent matrix of transition probabiltiies p(x_t|z_t) for each x- and z-states
        xs_sequence: Union[List, np.ndarray]
            Full list of x-states for which to compute the probability.
            The order must be [x_T, x_{T-1},..., x_1]

    Returns:
        float: The probability of the input sequence.
    '''
    # xs_sequence must be ordered as [x_T, x_{T-1},..., x_1]
    A = np.array(A)
    B = np.array(B)
    xs_sequence = np.array(xs_sequence)

    assert A.shape[0] == A.shape[1], 'A must be a square matrix'
    assert A.shape[0] == B.shape[0], 'number of rows of A and B must be the same'
    
    pi = np.array(pi).reshape(pi.size, 1)
    assert pi.shape[0] == B.shape[0], 'number of rows of pi and B must be the same'
    # x_t states can go from 1 to K
    assert (xs_sequence <= B.shape[1]).all(), 'there are states in input sequence that go beyond dimensions of B'
    
    T_ = len(xs_sequence)
    M_ = B.shape[0]
    K_ = B.shape[1]

    # compute initial state
    alpha_ini = alpha_initial(pi, B)
    # get sequence of alphas for arbitrary xs
    alpha_list = alpha_recurrent(A, B, T_, alpha_ini)
    # get general probability for last alpha from list
    # dim(general_prob) = (K,K,...K)
    general_prob = get_general_probability_from_alpha(alpha_list[-1])
    # indices of general_prob start at 0 but x states are indexed by 1,..K
    # thats why we use xs_sequence-1. 
    # pick x-index for each K-dimension
    return general_prob[tuple(xs_sequence-1)]


#### general check

Lets see if there is some general error in the functions

In [4]:
alpha1 = alpha_initial(pi, B)
alpha_l = alpha_recurrent(A, B, 3, alpha1)

In [5]:
alpha_l[-1].shape

(3, 2, 2, 2)

In [6]:
get_probability_sequence(pi, A, B, [1,2,1,1,2,2])

0.0012056062545468752

In [7]:
print(alpha_l[1].shape) # M x K (x_2) x K (x_1)
print(alpha_l[2].shape) # M x K (x_3) x K (x_2) x K (x_1)

(3, 2, 2)
(3, 2, 2, 2)


In [56]:
alpha1 = alpha_initial(pi, B)
alpha2 = np.einsum('ik,jl,js->ikls', B, alpha1, A)
alpha2_d = np.diagonal(alpha2, axis1=0, axis2=3)

### particular check

Lets compare the result with ones obtained by hand

In [5]:
# start from alpha_1
alpha1 = alpha_initial(pi, B)
alpha1

array([[0.01 , 0.09 ],
       [0.035, 0.065],
       [0.   , 0.8  ]])

In [ ]:
# alpha2 = B(z_2, x_2) \sum_z alpha1(t=1, z) A(z_1,z_2)